In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup

In [59]:
url = 'https://giffords.org/lawcenter/resources/scorecard2020/#rankings'


In [62]:
#Make a get request to retrieve the page
html_page = requests.get(url) 

#Pass the page contents to beautiful soup for parsing
soup = BeautifulSoup(html_page.content, 'html.parser') 

# States use the 'strong' tag
tables = soup.find_all('table')


In [63]:
state_rankings_2020 = {'Year': [], 'State':[], 'Grade': [], 'State_GPA': [], 'Gun_Death_Rate': []}
# Define grading scale
grades = {'A':4, 'A-': 3.7, 
          'B+': 3.3, 'B': 3, 'B-':2.7, 
          'C+': 2.3, 'C': 2, 'C-': 1.7, 
          'D+': 1.3, 'D': 1, 'D-': .7, 'F': 0}

# Iterate through each row of table and save to dictionary
for i in range(50):
    state_rankings_2020['Year'].append(2020)
    state_rankings_2020['State'].append(tables[0].find_all('td')[5 * i + 1].text)
    state_rankings_2020['Grade'].append(tables[0].find_all('td')[5 * i + 2].text)
    state_rankings_2020['State_GPA'].append(grades[state_rankings_2020['Grade'][-1]])
    state_rankings_2020['Gun_Death_Rate'].append(float(tables[0].find_all('td')[5 * i + 4].text))

# Create dataframe from dictionary
pd.DataFrame.from_dict(state_rankings_2020).to_csv('./giffords_2020.csv')

In [64]:
state_rankings_2020 = pd.read_csv('./giffords_2020.csv', index_col=0)
state_rankings_2020.head()

,Year,State,Grade,State_GPA,Gun_Death_Rate
0,2020,Alabama,F,0.0,22.17
1,2020,Alaska,F,0.0,24.49
2,2020,Arizona,F,0.0,15.05
3,2020,Arkansas,F,0.0,19.26
4,2020,California,A,4.0,7.21


In [21]:
state_rankings_2020.describe()

,Year,State_GPA,Gun_Death_Rate
count,50.0,50.000000,50.000000
mean,2020.0,1.448000,13.413600
std,0.0,1.470699,5.458588
min,2020.0,0.000000,3.390000
25%,2020.0,0.000000,10.300000
50%,2020.0,1.000000,12.735000
75%,2020.0,2.300000,16.410000
max,2020.0,4.000000,24.490000


In [65]:
from nltk.stem import WordNetLemmatizer 
LEMMER = WordNetLemmatizer() 

from nltk.stem.porter import PorterStemmer
STEMMER=PorterStemmer()

def MY_STEMMER(str_input):
    words = re.sub(r"[^A-Za-z\-]", " ", str_input).lower().split()
    words = [STEMMER.stem(word) for word in words]
    return words

def MY_LEMMER(str_input):
    words = re.sub(r"[^A-Za-z\-]", " ", str_input).lower().split()
    words = [LEMMER.lemmatize(word) for word in words]
    return words

In [66]:
from newsapi import NewsApiClient
import os
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
import re
current_directory = os.getcwd()


In [67]:
topics = ['gun ownership', 'mass shooting', 'hunting', 'second amendment','freedom']


In [68]:
newsapi = NewsApiClient(api_key='76acda18a67042c3b2e0568a00b9d8cf')

#within the current folder, this code creates a new folder called corpus and then makes 
# one folder for each of the topic and stores the respective files within them.
for topic in topics:
    all_articles = newsapi.get_everything(q=topic,
                                      page_size=100)
    corpus_directory = os.path.join(current_directory, "corpus")
    if not os.path.exists(corpus_directory):
        os.makedirs(corpus_directory)
    corpus_directory = os.path.join(corpus_directory, topic)
    if not os.path.exists(corpus_directory):
        os.makedirs(corpus_directory)
    file_count = 0
    for article in all_articles['articles']:
        description = article['description']
        file_count += 1
        filename= topic+str(file_count)+".txt"
        MyFile=open(corpus_directory + "/" + filename,"w+")
        MyFile.write(description)
        MyFile.close()


In [69]:
corpus_path=os.path.join(current_directory, "corpus")
## Empty List 
all_file_names = [] 
CorpusNameList=os.listdir(corpus_path) 
ListOfCompleteFiles=[] 
MyLables = []
for folder_name in CorpusNameList: 
    next_corpus=corpus_path+"/"+str(folder_name) 
    for fname in os.listdir(next_corpus): 
            full_path=next_corpus+"/"+fname 
            ListOfCompleteFiles.append(full_path) 
            MyLables.append(folder_name)

In [39]:
# MyVect_CV=CountVectorizer(input="filename", stop_words="english", max_features=100) 

# Vect_CV = MyVect_CV.fit_transform(ListOfCompleteFiles) 
# ColumnNamesCV=MyVect_CV.get_feature_names_out() 
# CorpusDF_CV=pd.DataFrame(Vect_CV.toarray(),columns=ColumnNamesCV)
# for nextcol in CorpusDF_CV.columns: 
#     if(re.search(r'[^A-Za-z]+', nextcol)): 
#         CorpusDF_CV= CorpusDF_CV.drop([nextcol], axis=1)
# CorpusDF_CV.insert(0, 'LABEL', MyLables)
# print(CorpusDF_CV)

In [70]:
# using the regular expression r'\b[a-zA-Z]{4,10}\b' to drop the tokens instead on dropping them manually.
My_TF1=TfidfVectorizer(input="filename", stop_words="english",tokenizer=MY_STEMMER,token_pattern = r'\b[a-zA-Z]{4,10}\b', max_features=100) 

X_TF1 = My_TF1.fit_transform(ListOfCompleteFiles) 


ColNames=My_TF1.get_feature_names_out()
CorpusDF_TF=pd.DataFrame(X_TF1.toarray(), columns=ColNames)
CorpusDF_TF.insert(loc=0, column='LABEL', value=MyLables)
print(CorpusDF_TF)

                LABEL    -  -year-old  accord     amend  american  appear  \
0    second amendment  0.0        0.0     0.0  0.000000  0.000000     0.0   
1    second amendment  0.0        0.0     0.0  0.436849  0.436849     0.0   
2    second amendment  0.0        0.0     0.0  0.000000  0.000000     0.0   
3    second amendment  0.0        0.0     0.0  0.000000  0.000000     0.0   
4    second amendment  0.0        0.0     0.0  0.000000  0.000000     0.0   
..                ...  ...        ...     ...       ...       ...     ...   
495           freedom  0.0        0.0     0.0  0.000000  0.000000     0.0   
496           freedom  0.0        0.0     0.0  0.000000  0.000000     0.0   
497           freedom  0.0        0.0     0.0  0.000000  0.000000     0.0   
498           freedom  0.0        0.0     0.0  0.000000  0.000000     0.0   
499           freedom  0.0        0.0     0.0  0.000000  0.000000     0.0   

     assault  attack  ban  ...    u  unit       use  violenc        wa  \
0

/home/akhil/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  

In [72]:
from GoogleNews import GoogleNews
googlenews = GoogleNews(lang='en', region='US',start='01/01/2023',end='02/01/2023')

In [77]:
all_google_results = []
all_google_labels = []
for topic in topics:
    googlenews.get_news(topic)
    google_results = googlenews.get_texts()
    all_google_results.extend(google_results)
    all_google_labels.extend([topic]*len(google_results))

https://news.google.com/search?q=gun%2Bownership&hl=en
https://news.google.com/search?q=mass%2Bshooting&hl=en
https://news.google.com/search?q=hunting&hl=en
https://news.google.com/search?q=second%2Bamendment&hl=en
https://news.google.com/search?q=freedom&hl=en


In [78]:
# google_results = googlenews.get_texts()
# google_results
# AllLabelsList.append([topic]*len(google_results))
# #topics = ['gun ownership', 'mass shooting', 'hunting', 'second amendment','freedom']
all_google_results

['Most Gun Laws Aren’t Backed Up By Evidence. Here’s Why.',
 'UN criticizes Israeli plan to ease gun ownership requirements after terror attacks',
 'Federal judge in Oklahoma rules against ban on gun ownership for marijuana users',
 'Connecticut leaders, advocates to hold rally against federal gun ownership ruling',
 'Federal judge in Oklahoma rules against ban on gun ownership for marijuana users',
 "Mental health checks for WA's gun owners to become mandatory under changes to firearms laws",
 'Under Bolsonaro, gun ownership rose, killings fell, Brazil debates why',
 'Florida bill could hand gun owners huge win, make the US a constitutional carry-majority country',
 'California Has More Than 100 Gun Laws. Why Don’t They Stop More Mass Shootings?',
 'Court Strikes Down Ban On Gun Ownership For Certain Domestic Violence Offenders',
 'Funny Papers Again Column | America as Hostage: But Who Holds the Gun?',
 'Ban on Gun Possession by Cannabis Users Is Unconstitutional ...',
 "Supreme Cour

In [79]:
My_TF1_GN=TfidfVectorizer(input='content',
                       stop_words='english',
                       token_pattern = r'\b[a-zA-Z]{4,10}\b',
                       max_features=1000)
X_TF1_GN=My_TF1_GN.fit_transform(all_google_results)
ColNames_GN=My_TF1_GN.get_feature_names_out()
CorpusDF_TF_GN=pd.DataFrame(X_TF1_GN.toarray(), columns=ColNames_GN)
CorpusDF_TF_GN.insert(loc=0, column='LABEL', value=all_google_labels)
print(CorpusDF_TF_GN)

              LABEL  abusers  academic  access  accident  action  activists  \
0     gun ownership      0.0  0.000000     0.0       0.0     0.0        0.0   
1     gun ownership      0.0  0.000000     0.0       0.0     0.0        0.0   
2     gun ownership      0.0  0.000000     0.0       0.0     0.0        0.0   
3     gun ownership      0.0  0.000000     0.0       0.0     0.0        0.0   
4     gun ownership      0.0  0.000000     0.0       0.0     0.0        0.0   
...             ...      ...       ...     ...       ...     ...        ...   
4009        freedom      0.0  0.000000     0.0       0.0     0.0        0.0   
4010        freedom      0.0  0.000000     0.0       0.0     0.0        0.0   
4011        freedom      0.0  0.735959     0.0       0.0     0.0        0.0   
4012        freedom      0.0  0.000000     0.0       0.0     0.0        0.0   
4013        freedom      0.0  0.000000     0.0       0.0     0.0        0.0   

      address  adopt  adults  ...  wounded  wounds 

In [ ]:
links = [
    "https://giffords.org/lawcenter/gun-laws/second-amendment/the-supreme-court-the-second-amendment/",
    "https://www.americas1stfreedom.org/articles/2018/10/22/guns-made-america-great/",
    "https://www.vpc.org/studies/unsafe.htm"
]

In [80]:
response = requests.get("https://giffords.org/lawcenter/gun-laws/second-amendment/the-supreme-court-the-second-amendment/")
MyTXT=response.text

In [88]:
# MyTXT

In [87]:
# MyList=MyTXT.split(" ")

# MyVectLDA=CountVectorizer(input='content', stop_words="english")
# data_vectorized = MyVectLDA.fit_transform(MyList)
# ColumnNamesLDA=MyVectLDA.get_feature_names()
# #print(ColumnNamesLDA)
# FinalDF=pd.DataFrame(data_vectorized.toarray(),columns=ColumnNamesLDA)

# RemoveWords=["href", "class", "div", "block", "nav", "pagination", 
#              "space", "https", "http", "template", "squarespace", "id", "jpg", "image",
#              "header", "function", "folder"]


# for nextcol in FinalDF.columns:
#     if(re.search(r'[^A-Za-z]+', nextcol)):
#         #print(nextcol)
#         FinalDF=FinalDF.drop([nextcol], axis=1)
#     elif(len(str(nextcol))<6):
#         FinalDF=FinalDF.drop([nextcol], axis=1)
#     elif(len(str(nextcol))>15):
#         FinalDF=FinalDF.drop([nextcol], axis=1)
#     elif(nextcol in RemoveWords):
#         FinalDF=FinalDF.drop([nextcol], axis=1)
        

# print(FinalDF)

In [89]:
# for nextcol in FinalDF.columns:
#     if(re.search(r'[^A-Za-z]+', nextcol)):
#         #print(nextcol)
#         FinalDF=FinalDF.drop([nextcol], axis=1)
#     elif(len(str(nextcol))<6):
#         FinalDF=FinalDF.drop([nextcol], axis=1)
#     elif(len(str(nextcol))>15):
#         FinalDF=FinalDF.drop([nextcol], axis=1)
#     elif(nextcol in RemoveWords):
#         FinalDF=FinalDF.drop([nextcol], axis=1)
# FinalDF.head()

In [91]:
a = CorpusDF_TF_GN
b = CorpusDF_TF

In [93]:
final_df = pd.merge(CorpusDF_TF, CorpusDF_TF_GN, on="LABEL")


In [94]:
final_df.describe()

,-,-year-old,accord,amend,american_x,appear,assault_x,attack_x,ban,california_x,...,wounded,wounds,wreckage,wyndham,wyoming,year_y,years,york,young,zelensky
count,401400.000000,401400.000000,401400.000000,401400.000000,401400.000000,401400.000000,401400.000000,401400.000000,401400.000000,401400.000000,...,401400.000000,401400.000000,401400.000000,401400.000000,401400.000000,401400.000000,401400.000000,401400.000000,401400.000000,401400.000000
mean,0.009795,0.011594,0.013230,0.023265,0.019361,0.010619,0.005755,0.013048,0.016492,0.028209,...,0.003141,0.000963,0.001893,0.000868,0.001377,0.010661,0.004527,0.002010,0.001707,0.000789
std,0.069926,0.068695,0.087136,0.106172,0.091395,0.073983,0.045216,0.078175,0.082154,0.089015,...,0.038386,0.020311,0.029968,0.019432,0.025300,0.066377,0.056355,0.032823,0.024784,0.017655
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.714916,0.783695,1.000000,0.874761,0.611157,0.760489,0.498411,0.773207,0.825743,0.480185,...,0.534762,0.429422,0.500633,0.435709,0.503259,0.691525,0.894685,0.652402,0.379078,0.395873


In [95]:
final_df

,LABEL,-,-year-old,accord,amend,american_x,appear,assault_x,attack_x,ban,...,wounded,wounds,wreckage,wyndham,wyoming,year_y,years,york,young,zelensky
0,second amendment,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,second amendment,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,second amendment,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,second amendment,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,second amendment,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401395,freedom,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
401396,freedom,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.894685,0.0,0.0,0.0
401397,freedom,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
401398,freedom,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
